In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
import os
import mne
import random
from sklearn.metrics import classification_report

In [57]:
MDD_dataset_Filenames = os.listdir('./data/MDD')
H_dataset_Filenames = os.listdir('./data/H')
MDD_dataset_list = []
H_dataset_list = []

for filename in MDD_dataset_Filenames:
    data = mne.io.read_raw_edf('./dataset/EC/' + str(filename))
    MDD_dataset_list.append(data)  
    
for filename in H_dataset_Filenames:
    data = mne.io.read_raw_edf('./dataset/EC/' + str(filename))
    H_dataset_list.append(data)  



Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S1 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S10 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S11 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S12 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S13 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S14 EC.edf...
EDF file detected
Setting channel 

In [58]:
MDD_dataframe_list = [] #MDD 환자의 데이터프레임 리스트
H_dataframe_list = [] #건강한 사람의 데이터프레임 리스트
for raw in MDD_dataset_list:
    raw_data = raw.get_data(return_times=True)
    channels = raw.ch_names
    T_data = np.transpose(raw_data[0])
    time = np.transpose(raw_data[1])
    temp = pd.DataFrame(T_data, columns = channels, index = time)
    MDD_dataframe_list.append(temp)
    
for raw in H_dataset_list:
    raw_data = raw.get_data(return_times=True)
    channels = raw.ch_names
    T_data = np.transpose(raw_data[0])
    time = np.transpose(raw_data[1])
    temp = pd.DataFrame(T_data, columns = channels, index = time)
    H_dataframe_list.append(temp)

MDD_dataframe_list[23].rename(columns = {'E F3-Ref': 'F3'}, inplace = True)

In [91]:
X = pd.DataFrame(columns=range(45))
y = []
for i in range(len(MDD_dataframe_list)):
    a = MDD_dataframe_list[i]
    #데이터 클린징
    a.F3 = a.F3[a.F3 > -0.0005]
    a = a.fillna(method='ffill')
    X[i] = a.F3
    y.append(1)
    
for i in range(30, 30+len(H_dataframe_list)):
    a = H_dataframe_list[30-i]
    a.rename(columns = {'EEG F3-LE' : 'F3'}, inplace = True)
    #데이터 클린징
    a.F3 = a.F3[a.F3 > -0.0005]
    a = a.fillna(method='ffill')
    X[i] = a.F3
    y.append(0)
    
X = X.fillna(method='ffill')
X = X.T
X['label'] = y

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
27    1
28    1
29    1
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
Name: label, dtype: int64

In [94]:
# 타겟 컬럼과 특징 컬럼 정의
target_col = 'label'
target = X[target_col]
features = X[X.columns.difference(['label'])]


AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [97]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)


3     1
17    1
1     1
8     1
25    1
4     1
30    0
27    1
28    1
19    1
20    1
39    0
42    0
10    1
38    0
7     1
11    1
36    0
44    0
24    1
9     1
40    0
18    1
15    1
41    0
14    1
29    1
12    1
23    1
33    0
32    0
37    0
22    1
13    1
26    1
31    0
Name: label, dtype: int64

In [99]:
from sklearn.svm import SVC
#SVM Model
svm_linear = SVC(kernel='linear', random_state=0)
svm_linear.fit(X_train, y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [101]:
#SVM 선형 평가
print("테스트 세트 예측 : ", svm_linear.predict(X_test))
print("테스트 세트 정확도 : ", svm_linear.score(X_test, y_test))

y_pred = svm_linear.predict(X_test)
print(classification_report(y_test, y_pred))


테스트 세트 예측 :  [1 1 1 1 1 1 1 1 1]
테스트 세트 정확도 :  0.6666666666666666
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.67      1.00      0.80         6

    accuracy                           0.67         9
   macro avg       0.33      0.50      0.40         9
weighted avg       0.44      0.67      0.53         9



C:\Users\mws10\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [102]:
from sklearn.svm import SVC
#SVM Model
svm_rbf = SVC(kernel='rbf', random_state=0)
svm_rbf.fit(X_train, y_train)


C:\Users\mws10\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [103]:
#SVM 선형 평가
print("테스트 세트 예측 : ", svm_rbf.predict(X_test))
print("테스트 세트 정확도 : ", svm_rbf.score(X_test, y_test))

y_pred = svm_rbf.predict(X_test)
print(classification_report(y_test, y_pred))

테스트 세트 예측 :  [1 1 1 1 1 1 1 1 1]
테스트 세트 정확도 :  0.6666666666666666
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.67      1.00      0.80         6

    accuracy                           0.67         9
   macro avg       0.33      0.50      0.40         9
weighted avg       0.44      0.67      0.53         9



C:\Users\mws10\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([array([ 1.90800000e-03+0.00000000e+00j, -3.47999914e-04-6.56490085e-05j,
       -4.03775530e-05+2.08309585e-04j, ...,
       -3.35264510e-04-1.25124121e-04j, -4.03775530e-05-2.08309585e-04j,
       -3.47999914e-04+6.56490085e-05j]),
       array([ 0.006986  +0.j        , -0.00330351+0.00533055j,
       -0.00477856-0.00129966j, ...,  0.00136779-0.00034316j,
       -0.00477856+0.00129966j, -0.00330351-0.00533055j]),
       array([ 1.60000000e-03+0.00000000e+00j, -3.03536698e-05-5.38778943e-05j,
        5.70356802e-05+1.64448746e-04j, ...,
       -1.81983017e-04-6.14077262e-05j,  5.70356802e-05-1.64448746e-04j,
       -3.03536698e-05+5.38778943e-05j]),
       array([ 2.56000000e-03+0.j        , -3.38581229e-04-0.00021692j,
        1.29294746e-05+0.00019567j, ..., -1.77079729e-04+0.00016408j,
        1.29294746e-05-0.00019567j, -3.38581229e-04+0.00021692j]),
       array([1.42500000e-03+0.00000000e+00j, 1.21781219e-04+9.16660436e-05j,
       1.29545166e-04-5.30974541e-05j, ...,
    